In [ ]:
import os
import re

folder_path = "/content/공감"
file_list = os.listdir(folder_path)

# .json 확장자를 유지하면서 'Empathy_숫자.json' 형태로 바꾸는 정규식
pattern = re.compile(r'(Empathy).*?(\d+).*?(\.json)$')

for file_name in file_list:
    match = pattern.search(file_name)
    if match:
        new_name = f"{match.group(1)}_{match.group(2)}{match.group(3)}"
        src = os.path.join(folder_path, file_name)
        dst = os.path.join(folder_path, new_name)
        os.rename(src, dst)
        print(f"Renamed: {file_name} → {new_name}")


In [ ]:
import os
import re

folder_path = "/content/공감2"
file_list = os.listdir(folder_path)

pattern = re.compile(r'(Empathy).*?(\d+).*?(\.json)$')

for file_name in file_list:
    match = pattern.search(file_name)
    if match:
        new_name = f"{match.group(1)}_{match.group(2)}{match.group(3)}"
        src = os.path.join(folder_path, file_name)
        dst = os.path.join(folder_path, new_name)
        os.rename(src, dst)
        print(f"Renamed: {file_name} → {new_name}")


In [ ]:
import os
import re

folder_path = "/content/공감"

# 숫자를 기준으로 정렬하는 함수 정의
def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(1)) if match else float('inf')

# 파일 리스트 가져와서 숫자 기준으로 정렬
file_list = sorted(os.listdir(folder_path), key=extract_number)

# 정렬된 파일 출력
for file_name in file_list:
    print(file_name)


In [ ]:
import os
import re

folder_path = "/content/공감2"

# 숫자를 기준으로 정렬하는 함수 정의
def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(1)) if match else float('inf')

# 파일 리스트 가져와서 숫자 기준으로 정렬
file_list = sorted(os.listdir(folder_path), key=extract_number)

# 정렬된 파일 출력
for file_name in file_list:
    print(file_name)


In [ ]:
import os
import json

folder_path = "/content/공감"
text_list = []

# Empathy_숫자.json만 대상으로 처리
for filename in sorted(os.listdir(folder_path)):
    if filename.startswith("Empathy_") and filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            for utterance in data.get("utterances", []):
                text = utterance.get("text")
                if text:  # text가 None이 아닌 경우만 추가
                    text_list.append(text)

# 결과 확인
print(f"총 {len(text_list)}개의 문장이 추출되었습니다.")
print(text_list[:5])  # 앞의 일부만 확인


In [ ]:
import os
import json

folder_path = "/content/공감2"
# Empathy_숫자.json만 대상으로 처리
for filename in sorted(os.listdir(folder_path)):
    if filename.startswith("Empathy_") and filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            for utterance in data.get("utterances", []):
                text = utterance.get("text")
                if text:  # text가 None이 아닌 경우만 추가
                    text_list.append(text)

# 결과 확인
print(f"총 {len(text_list)}개의 문장이 추출되었습니다.")
print(text_list[:5])  # 앞의 일부만 확인


In [ ]:
len(text_list)

In [ ]:
import json

# 리스트를 JSON 파일로 저장
output_path = "/content/text_data.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(text_list, f, ensure_ascii=False, indent=4)

print(f"텍스트 리스트가 JSON 파일로 저장되었습니다: {output_path}")

In [ ]:
import json
import os

data_dir = "/content/공감"  # JSON 파일들이 있는 폴더
output_data = []

for filename in os.listdir(data_dir):
    if filename.endswith(".json"):
        file_path = os.path.join(data_dir, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        dialogue = []
        for utt in data.get("utterances", []):
            role = "user" if utt["role"] == "speaker" else "assistant"
            dialogue.append({
                "role": role,
                "text": utt["text"]
            })

        if dialogue:
            output_data.append({"dialogue": dialogue})

In [ ]:
data_dir = "/content/공감2"  # JSON 파일들이 있는 폴더

for filename in os.listdir(data_dir):
    if filename.endswith(".json"):
        file_path = os.path.join(data_dir, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        dialogue = []
        for utt in data.get("utterances", []):
            role = "user" if utt["role"] == "speaker" else "assistant"
            dialogue.append({
                "role": role,
                "text": utt["text"]
            })

        if dialogue:
            output_data.append({"dialogue": dialogue})

In [ ]:
for dialogue_data in output_data:
    for turn in dialogue_data["dialogue"]:
        # role이 "assistant"일 때 text 앞에 공백 추가
        if turn["role"] == "assistant":
            if not turn["text"].startswith(" "):
                turn["text"] = " " + turn["text"]


In [ ]:
training_pairs = []

for dialogue_data in output_data:
    dialogue = dialogue_data["dialogue"]
    for i in range(len(dialogue) - 1):
        if dialogue[i]["role"] == "user" and dialogue[i+1]["role"] == "assistant":
            prompt = dialogue[i]["text"]
            completion = dialogue[i+1]["text"]
            if not completion.startswith(" "):
                completion = " " + completion
            training_pairs.append({
                "prompt": prompt,
                "completion": completion
            })

# 확인용 출력
for pair in training_pairs[:3]:
    print(pair)


In [ ]:
training_pairs2 = []

for i in range(0, len(training_pairs), 2):
    # i: user 발화
    # i+1: assistant 발화
    if i+1 < len(training_pairs):
        training_pairs2.append({
            "completion": " " + training_pairs[i]["prompt"],        # user 발화가 completion
            "prompt": training_pairs[i+1]["completion"].lstrip()  # assistant 발화가 prompt
        })


In [ ]:
import json

# 두 리스트 합치기
training = training_pairs + training_pairs2

In [ ]:
# JSON 파일로 저장
with open('training.json', 'w', encoding='utf-8') as f:
    json.dump(training, f, ensure_ascii=False, indent=2)


In [ ]:
import json

# 기존 training.json 로드
with open("training.json", "r", encoding="utf-8") as f:
    training_data = json.load(f)

# 새로운 형태로 변환
converted_data = []
for item in training_data:
    prompt = item["prompt"].strip()
    completion = item["completion"].strip()
    combined_text = f"어르신: {prompt}\n모델: {completion}\n"
    converted_data.append({"text": combined_text})

# 새 파일로 저장
with open("training_text_format.json", "w", encoding="utf-8") as f:
    json.dump(converted_data, f, ensure_ascii=False, indent=2)


In [ ]:
import json

# 1. 파일 경로
input_path = "/content/training_text_format.json"
output_path = "/content/converted_dataset.json"

# 2. 파일 읽기
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 3. 변환
converted_data = []
for item in data:
    text = item["text"]

    # "어르신:"과 "모델:" 기준으로 나누기
    if "어르신:" in text and "모델:" in text:
        try:
            input_part = text.split("어르신:")[1].split("모델:")[0].strip()
            output_part = text.split("모델:")[1].strip()

            converted_data.append({
                "input": f"어르신: {input_part}",
                "output": f"모델: {output_part}"
            })
        except Exception as e:
            print(f"오류 발생한 데이터: {text}")
            continue

# 4. 새 파일로 저장
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(converted_data, f, ensure_ascii=False, indent=2)

print(f"저장 위치 : {output_path}")


In [ ]:
import json

# 파일 경로
input_path = "/content/converted_dataset.json"
output_path = "/content/cleaned_dataset.json"

# 파일 불러오기
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 접두어 제거
for item in data:
    if item["input"].startswith("어르신:"):
        item["input"] = item["input"].replace("어르신:", "", 1).strip()
    if item["output"].startswith("모델:"):
        item["output"] = item["output"].replace("모델:", "", 1).strip()

# 저장
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"저장 위치 : {output_path}")


In [ ]:
import json

with open("/content/cleaned_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"총 항목 수: {len(data)}")


In [ ]:
import json

# 원본 파일 경로
input_path = "/content/cleaned_dataset.json"
output_path = "/content/cleaned_dataset_half.json"

# 데이터 로드
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 절반 잘라내기
half_length = len(data) // 2  # 정수 나눗셈으로 6353
cut_data = data[:half_length]

# 저장
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(cut_data, f, ensure_ascii=False, indent=2)

print(f"저장 위치 : {output_path}")
